In [131]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import ShuffleSplit
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import AdaBoostClassifier

In [132]:
df = pd.read_csv('tash-pt.csv')
df

,id_twitter,sentiment,text
0,1067856049821155334,1,@tchaugip @n00bona História vence técnica faci...
1,1070902848957964288,1,@camandis aa obrigado perfeito o seu timing ti...
2,1073158858925838336,0,moldura artística favorita entre filistinos al...
3,1073220881344856064,0,Minha cama tá no localizada no meio nesse mome...
4,1073347140334833664,0,@eueduramos o mundo so vai mudar quando as pes...
...,...,...,...
2782,1068599890761129985,-1,@PastorMalafaia Concordo com Prof Olavo de Car...
2783,1071376463230840833,-1,@ClaudiaLeitte se o clipe de saudade não for g...
2784,1073978880787181568,1,agr o ar vai ficar ligado 24 por 48
2785,1074925265980145666,0,e eu que no meio de tantos problemas ainda con...


In [133]:
df.drop('id_twitter', inplace = True, axis = 1) #deletando coluna 'id_twitter'
df.head()

,sentiment,text
0,1,@tchaugip @n00bona História vence técnica faci...
1,1,@camandis aa obrigado perfeito o seu timing ti...
2,0,moldura artística favorita entre filistinos al...
3,0,Minha cama tá no localizada no meio nesse mome...
4,0,@eueduramos o mundo so vai mudar quando as pes...


In [134]:
df.isnull().sum()

sentiment    0
text         0
dtype: int64

In [135]:
df.drop_duplicates() #deletando também duplicatas, se existirem
df.shape

(2787, 2)

In [136]:
df['sentiment'].value_counts()

 0    1018
 1     888
-1     881
Name: sentiment, dtype: int64

In [137]:
print(df['text'][0])

@tchaugip @n00bona História vence técnica facilmente. A menos que seja algo experimental que traga algo muito diferente e novo, pois aí agrega ao panteao de técnicas e então é lembrado. Caso contrário é só execução.


## Pré-processamento do texto

**Limpeza do texto**

Excluindo os nomes de usuários

In [138]:
import re

print (df['text'][1])
texto = df['text'][1]

res = re.sub("@[^ ]+", "", texto)

print(res)

@camandis aa obrigado perfeito o seu timing tinha acabado de dar 18 minutos ja tirei
 aa obrigado perfeito o seu timing tinha acabado de dar 18 minutos ja tirei


In [139]:
def user_clean(text):
    res = re.sub("@[^ ]+", "", text)
    return res

for i in df.index:
    texto = df['text'][i]
    df['text'][i] = user_clean(texto)

df

<ipython-input-139-1e0eb6510cf5>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][i] = user_clean(texto)


,sentiment,text
0,1,História vence técnica facilmente. A menos q...
1,1,aa obrigado perfeito o seu timing tinha acaba...
2,0,moldura artística favorita entre filistinos al...
3,0,Minha cama tá no localizada no meio nesse mome...
4,0,o mundo so vai mudar quando as pessoas entend...
...,...,...
2782,-1,Concordo com Prof Olavo de Carvalho. O Escola...
2783,-1,se o clipe de saudade não for gravado no Pelo...
2784,1,agr o ar vai ficar ligado 24 por 48
2785,0,e eu que no meio de tantos problemas ainda con...


Removendo acentos e caracteres especiais

In [140]:
import unicodedata

def f_clean(df, text):
    df['text'] = df['text'].replace(regex='[!/,.-]',value='').apply(lambda x: unicodedata.normalize('NFKD', x).encode('ascii','ignore').decode("utf-8"))
    return df
                                                                    
f_clean(df, df['text'])
print('\n',df['text'][0])
print('\n',df['text'][9])
print('\n',df['text'][1575])
print('\n',df['text'][2019])
print('\n',df['text'][1981])


   Historia vence tecnica facilmente A menos que seja algo experimental que traga algo muito diferente e novo pois ai agrega ao panteao de tecnicas e entao e lembrado Caso contrario e so execucao

 viado eu to apaixonado pelo   nao da amo um artista

 se eu tivesse um canal no youtube c um milhao de inscritos ja fazer pelo menos cada inscrito me mandar um real por mes

 To rindo mais to preocupado joguei dentro denovo pqp tava na onda do boldo 

 Nao se preocupe com passado q ele sempre foi ausente
Se tava tudo igual eu faco ficar diferente 


## Remoção de stopwords

In [141]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

def f_stopwords(df, text):
    stop_words = stopwords.words('portuguese')
    df['text'] = df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
    return df

f_stopwords(df, df['text'])
print('\n',df['text'][0])
print('\n',df['text'][9])
print('\n',df['text'][1575])
print('\n',df['text'][2019])
print('\n',df['text'][1981])

[nltk_data] Downloading package stopwords to C:\Users\Letícia
[nltk_data]     Sousa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!



 Historia vence tecnica facilmente A menos algo experimental traga algo diferente novo pois ai agrega panteao tecnicas entao lembrado Caso contrario so execucao

 viado to apaixonado nao amo artista

 canal youtube c milhao inscritos ja fazer menos cada inscrito mandar real mes

 To rindo to preocupado joguei dentro denovo pqp tava onda boldo

 Nao preocupe passado q sempre ausente Se tava tudo igual faco ficar diferente


## Aplicação de stemming

depois ;)

## Vetorização

In [142]:
sample = df['text']
target = df['sentiment']
count_vect = CountVectorizer()
X_sample = count_vect.fit_transform(sample)
tfidf_transformer = TfidfTransformer()
X_sample_transform = tfidf_transformer.fit_transform(X_sample)

In [143]:
X, y = X_sample_transform, target
X.shape, y.shape

((2787, 11254), (2787,))

## K-Fold SVM

In [144]:
kf = KFold(n_splits=10, shuffle=True)

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    reg = svm.SVC(kernel = 'linear', C=1).fit(X_train, y_train)
    y_pred = reg.predict(X_test)
    print(reg.score(X_test, y_test))

0.46236559139784944
0.4551971326164875
0.43010752688172044
0.4551971326164875
0.4050179211469534
0.45878136200716846
0.4336917562724014
0.41007194244604317
0.460431654676259
0.43884892086330934


## K-Fold NB 

In [145]:
kf = KFold(n_splits=10, shuffle=True)

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    nb = MultinomialNB().fit(X_train, y_train)
    y_pred = nb.predict(X_test)
    print(nb.score(X_test, y_test))

0.4838709677419355
0.40860215053763443
0.43010752688172044
0.45878136200716846
0.44086021505376344
0.43727598566308246
0.4444444444444444
0.4676258992805755
0.44244604316546765
0.4712230215827338


## K-Fold LR

In [146]:
kf = KFold(n_splits=10, shuffle=True)

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    lr = LogisticRegression(max_iter=5000).fit(X_train, y_train)
    y_pred = lr.predict(X_test)
    print(lr.score(X_test, y_test))

0.4838709677419355
0.4731182795698925
0.4336917562724014
0.4910394265232975
0.4910394265232975
0.45161290322580644
0.3942652329749104
0.4136690647482014
0.4172661870503597
0.46402877697841727


## K-Fold AdaBoost

In [147]:
kf = KFold(n_splits=10, shuffle=True)

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    adb = AdaBoostClassifier().fit(X_train, y_train)
    y_pred = adb.predict(X_test)
    print(adb.score(X_test, y_test))

0.44802867383512546
0.4444444444444444
0.3942652329749104
0.4336917562724014
0.40860215053763443
0.4336917562724014
0.4014336917562724
0.4352517985611511
0.3776978417266187
0.4352517985611511


## K-Fold OneVsRest

In [148]:
kf = KFold(n_splits=10, shuffle=True)

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    ovr = OneVsRestClassifier(LinearSVC(random_state=0, max_iter=5000)).fit(X_train, y_train)
    y_pred = ovr.predict(X_test)
    print(ovr.score(X_test, y_test))

0.45161290322580644
0.43010752688172044
0.4157706093189964
0.4229390681003584
0.44086021505376344
0.4551971326164875
0.4121863799283154
0.4352517985611511
0.4568345323741007
0.4460431654676259
